# Yelp reviews sentiment analysis using pytorch

The purpose of this notebook is to go through all the basics for an NLP task. The breakdown of the tasks would be as follows:

1. **Data Processing**: Process the raw data, convert it to a pandas dataframe, and manipulate the data according to your need and save it to another csv file.
2. **Data Vectorization**: The process of converting the text reviews to a vector of intergers using one hot encoding. Deep learning models do not accept any textual inputs rather you need to feed the inputs as intgers or floats.
3. **Data vocabulary**: we need to create a vocabulary for an NLP task, because our model can learn only from the words it has seen so far and their position in the text as well. so for this purpose we need to know which word come how many times in a text, where it appear in the text. we store all such information in a python dictionary
4. **Data processing in pytorch**: We process the data in pytorch in using torch dataloader by input our dataset, batch_size. It automatically converts the dataset in batches of tensors for us. so we need not to split the dataset in batches separately. It also handles the autograd for us. In short we are missing out:
  * Batching the data
  * Shuffling the data
  * Load the data in parallel using multiprocessing workers.

Dataloader provides all these functions.

5. **Deep Learning Model**: so far the models that I a working on are:
  * Single Layer Perceptron with following params:
      * One Linear Layer of Softmax
      * Sigmoid activation unit
      * Adam optimizer to upgrade the weights of the parameters
      * Binary cross entropy loss which deals with models which spits binary outputs.

Explore some feed forward networks:
  * MLP: [TODO]
  * CNN: [TODO]
Explore Feedforward and feed backward networks such as:
  * BERT:[TODO]
  * RNN: [TODO]
  * LSTM: [TODO]
 

6. Training, Validation and Testing Loop.
7. Hyperparameters Tuning and their understanding.

[TODO]:
* Seed understanding as currently the output is changing [DONE]
* Use cuda in pytorch and re run the model : [DONE]
* Add code to store the vectorized data and model files.
* prediction of the model [DONE]
* other deep learning model implementation.




[NOTE]: One of the thing that I have observed so far is:

* If I use the very light dataset, then the simple perceptron works really well.
* If I use the full dataset of yelp reviews the perceptron overfits and I get accuracy of 100 percent, and I can clearly observe that it is overfitting because when I print top 20 positive words, it spits random garbage.










In [39]:
! git clone https://github.com/Gaurav-Pande/Sentiment-Analysis.git
! mv Sentiment-Analysis/data .
% cd data 
! ./get-all-data.sh
% cd ../
! ls

fatal: destination path 'Sentiment-Analysis' already exists and is not an empty directory.
mv: cannot stat 'Sentiment-Analysis/data': No such file or directory
/content/data
/content
data  sample_data  Sentiment-Analysis


In [0]:
import collections
import pandas as pd
import numpy as np
import re
from argparse import Namespace
import string

In [0]:
# defining the arguments for the yelp datasets
arg = Namespace(
    islite = False,
    train_csv_lite_with_split = "data/yelp/reviews_with_splits_lite.csv",
    train_csv = "data/yelp/raw_train.csv",
    test_csv = "data/yelp/raw_test.csv",
    train_split_ratio = 0.75,
    test_split_ratio = 0.25,
    seed = 1330,
    output_file_sentiment = "data/yelp/final_sentiment.csv",
    output_file_rating = "data/yelp/final_ratings.csv"
)

In [0]:
# lets read the raw data stored in args
if not arg.islite:
  train_csv  = pd.read_csv(arg.train_csv, header=None, names = ["rating","review"])
  test_csv = pd.read_csv(arg.test_csv, header=None, names=["rating","review"])
  train_csv = train_csv[~pd.isnull(train_csv.review)]
  test_csv = test_csv[~pd.isnull(test_csv.review)]

In [43]:
# see some rows of train
train_csv.head()

,rating,review
0,1,"Unfortunately, the frustration of being Dr. Go..."
1,2,Been going to Dr. Goldberg for over 10 years. ...
2,1,I don't know what Dr. Goldberg was like before...
3,1,I'm writing this review to give you a heads up...
4,2,All the food is great here. But the best thing...


In [0]:
def partition_dataset(train_csv, test_csv):
  # creating new train, validation and test sets
  by_ratings = collections.defaultdict(list)
  for _, row in train_csv.iterrows():
    by_ratings[row.rating].append(row.to_dict())
  final_list =[]
  seed = 1000
  np.random.seed(arg.seed)
  for _,  item_list in sorted(by_ratings.items()):
    np.random.shuffle(item_list)
    total_rows = len(item_list)
    total_train_required = int(arg.train_split_ratio*total_rows)
    total_test_required = int(arg.test_split_ratio*total_rows)
    # Give data point a split attribute
    for item in item_list[:total_train_required]:
        item['split'] = 'train'
    for item in item_list[total_train_required:total_train_required+total_test_required]:
        item['split'] = 'val'

    # Add to final list
    final_list.extend(item_list)
    for _, row in test_csv.iterrows():
      row_dict = row.to_dict()
      row_dict['split'] = 'test'
      final_list.append(row_dict)

    return final_list, pd.DataFrame(final_list)


def preprocess_data(text):
  if type(text) == float:
        print(text)
  text = text.lower()
  text = re.sub(r"([.,!?])", r" \1 ", text)
  text = re.sub(r"[^a-zA-Z.,!?]+", r" ", text)
  return text


In [0]:
final_list, final_list_df = partition_dataset(train_csv,test_csv)
final_list_df.review = final_list_df.review.apply(preprocess_data)

In [46]:
# see the split of data in the dataset
final_list_df.split.value_counts()
#final_list_df.split['train']


train    210000
val       70000
test      38000
Name: split, dtype: int64

In [0]:
# Change the rating to sentiment
# rating of 1 ==> negative sentiment
# rating of 2 ===> positive sentiment
final_list_sentiment = final_list_df.copy()
final_list_rating = final_list_df.copy()
final_list_sentiment['rating'] = final_list_sentiment.rating.apply({1: 'negative', 2: 'positive'}.get)
#final_list_rating['rating'] = final_list_rating.rating.apply({'negative':1, 'positive':2}.get)


In [48]:
# see the dataset after changing to sentiment
final_list_sentiment.head()

,rating,review,split
0,negative,used to come here a lot then i think the owner...,train
1,negative,got a notice for the preferred customer sale l...,train
2,negative,the burgers here are probably the best burgers...,train
3,negative,i am a road warrior . i am used to eating alon...,train
4,negative,i come to this place whenever i am in town for...,train


In [49]:
final_list_rating.head()

,rating,review,split
0,1,used to come here a lot then i think the owner...,train
1,1,got a notice for the preferred customer sale l...,train
2,1,the burgers here are probably the best burgers...,train
3,1,i am a road warrior . i am used to eating alon...,train
4,1,i come to this place whenever i am in town for...,train


In [0]:
# save the new dataframes to the csv file
final_list_rating.to_csv(arg.output_file_rating)
final_list_sentiment.to_csv(arg.output_file_sentiment)

In [0]:
class Vocabulary(object):
  "class to process text and extract the vocab for mapping"
  def __init__(self, token_to_idx = None, add_unk = True, unk_token='<UNK>'):
    if token_to_idx is None:
      token_to_idx = {}

    self._token_to_idx = token_to_idx
    self._idx_to_token = {idx:token for token, idx in self._token_to_idx.items()}
    self._add_unk = add_unk
    self._unk_token = unk_token
    self.unk_index = -1
    if add_unk:
      self.unk_index = self.add_token(unk_token)

  def to_serializable(self):
    "returns  a dictionary which can be serializable"
    return {'token_to_idx': self._token_to_idx,
            'add_unk': self._add_unk,
            'unk_token':self._unk_token}

  @classmethod
  def from_serializable(cls, contents):
    "instantiate a vocab from serialized dictionary"
    return cls(**contents)

  def add_token(self, token):
    "update the mapping dictionary based ont the token"
    if token in self._token_to_idx:
      index = self._token_to_idx[token]
    else:
      index = len(self._token_to_idx)
      self._token_to_idx[token] = index
      self._idx_to_token[index] = token

    return index


  def lookup_token(self, token):
    "retrieve the index based on the token from the mapping dictionary"
    # i=1
    # for k,v in self._token_to_idx.items():
    #   if i==10:
    #     break
    #   print(k,v)
    if self.unk_index >= 0:
      return self._token_to_idx.get(token, self.unk_index)
    else:
      #print(self._token_to_idx)
      if token not in self._token_to_idx:
        #print(self._token_to_idx.keys())
        return 0
      return self._token_to_idx[token]

  def lookup_index(self, index):
    "retrieve the token based on the index from the mapping dictionary"
    if index not in self._idx_to_token:
      raise KeyError("The provided index: %d is not in the vocab"% index)
    else:
      return self._idx_to_token[index]

  

  def __str__(self):
    return "<Vocabulary(size=%d)>" % len(self)


  def __len__(self):
    "length of the vocabulary"
    return len(self._token_to_idx)

In [0]:
class ReviewVectorizer(object):
  " Used the vocbulary class to convert the tokens into actual numerical vector"

  def __init__(self, review_vocab, rating_vocab):
    """
    review_vocab :  maps word to integer
    rating_vocab : maps class label to integer("negative/positive")
    """
    self.review_vocab = review_vocab
    self.rating_vocab = rating_vocab

  def vectorize(self, review):
    "vectorize a text review to one hot encoding"
    one_hot = np.zeros(len(self.review_vocab), dtype=np.float32)
    for token in review.split(" "):
      if token not in string.punctuation:
        one_hot[self.review_vocab.lookup_token(token)] = 1
    return one_hot
    # for CNNs
    # one_hot_matrix = np.zeros(size= (len(self.review_vocab),300), dtype=np.float32 )
    # for token in review.split(" "):
    #   if token not in string.punctuation:



  @classmethod
  def from_dataframe(cls, review_df, cutoff=25):
    "instantiate a vector for reviews directly from dataset dataframe"
    review_vocab = Vocabulary(add_unk=True)
    rating_vocab = Vocabulary(add_unk=False)
    for rating in sorted(set(review_df.rating)):
      rating_vocab.add_token(rating)
    
    # add top words if count > provided counts
    word_count = collections.Counter()
    for review in review_df.review:
      for word in review.split(" "):
        if word not in string.punctuation:
          word_count[word] += 1


    for word, count in word_count.items():
      if count > cutoff:
        review_vocab.add_token(word)
    return cls(review_vocab, rating_vocab)

    @classmethod
    def from_serializable(cls, contents):
      review_vocab = Vocabulary.from_serializable(contents['review_vocab'])
      rating_vocab = Vocabulary.from_serializable(contents['rating_vocab'])
      return cls(review_vocab=review_vocab, rating_vocab=rating_vocab)


    def to_serializable(self):
      return {'review_vocab' : self.review_vocab.to_serializable(),
              'rating_vocab' : self.rating_vocab.to_serializable()         
      }

In [0]:
from torch.utils.data import Dataset, DataLoader
class ReviewDataset(Dataset):
    def __init__(self, review_df, vectorizer):
        """
        Args:
            review_df (pandas.DataFrame): the dataset
            vectorizer (ReviewVectorizer): vectorizer instantiated from dataset
        """
        self.review_df = review_df
        self._vectorizer = vectorizer

        self.train_df = self.review_df[self.review_df.split=='train']
        self.train_size = len(self.train_df)

        self.val_df = self.review_df[self.review_df.split=='val']
        self.validation_size = len(self.val_df)

        self.test_df = self.review_df[self.review_df.split=='test']
        self.test_size = len(self.test_df)

        self._lookup_dict = {'train': (self.train_df, self.train_size),
                             'val': (self.val_df, self.validation_size),
                             'test': (self.test_df, self.test_size)}

        self.set_split('train')

    @classmethod
    def load_dataset_and_make_vectorizer(cls, review_csv):
        """Load dataset and make a new vectorizer from scratch
        
        Args:
            review_csv (str): location of the dataset
        Returns:
            an instance of ReviewDataset
        """
        review_df = pd.read_csv(review_csv)
        train_review_df = review_df[review_df.split=='train']
        return cls(review_df, ReviewVectorizer.from_dataframe(train_review_df))
    
    @classmethod
    def load_dataset_and_load_vectorizer(cls, review_csv, vectorizer_filepath):
        """Load dataset and the corresponding vectorizer. 
        Used in the case in the vectorizer has been cached for re-use
        
        Args:
            review_csv (str): location of the dataset
            vectorizer_filepath (str): location of the saved vectorizer
        Returns:
            an instance of ReviewDataset
        """
        review_df = pd.read_csv(review_csv)
        vectorizer = cls.load_vectorizer_only(vectorizer_filepath)
        return cls(review_df, vectorizer)

    @staticmethod
    def load_vectorizer_only(vectorizer_filepath):
        """a static method for loading the vectorizer from file
        
        Args:
            vectorizer_filepath (str): the location of the serialized vectorizer
        Returns:
            an instance of ReviewVectorizer
        """
        with open(vectorizer_filepath) as fp:
            return ReviewVectorizer.from_serializable(json.load(fp))

    def save_vectorizer(self, vectorizer_filepath):
        """saves the vectorizer to disk using json
        
        Args:
            vectorizer_filepath (str): the location to save the vectorizer
        """
        with open(vectorizer_filepath, "w") as fp:
            json.dump(self._vectorizer.to_serializable(), fp)

    def get_vectorizer(self):
        """ returns the vectorizer """
        return self._vectorizer

    def set_split(self, split="train"):
        """ selects the splits in the dataset using a column in the dataframe 
        
        Args:
            split (str): one of "train", "val", or "test"
        """
        self._target_split = split
        self._target_df, self._target_size = self._lookup_dict[split]

    def __len__(self):
        return self._target_size

    def __getitem__(self, index):
        """the primary entry point method for PyTorch datasets
        
        Args:
            index (int): the index to the data point 
        Returns:
            a dictionary holding the data point's features (x_data) and label (y_target)
        """
        row = self._target_df.iloc[index]

        review_vector = \
            self._vectorizer.vectorize(row.review)

        rating_index = \
            self._vectorizer.rating_vocab.lookup_token(row.rating)

        return {'x_data': review_vector,
                'y_target': rating_index}

    def get_num_batches(self, batch_size):
        """Given a batch size, return the number of batches in the dataset
        
        Args:
            batch_size (int)
        Returns:
            number of batches in the dataset
        """
        return len(self) // batch_size  

In [0]:
# Now generate minibatches from the data using pytorch dataloader class
def generate_batches(dataset, batch_size, shuffle=True,
                     drop_last=True, device="cpu"):
    """
    A generator function which wraps the PyTorch DataLoader. It will 
      ensure each tensor is on the write device location.
    """
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size,
                            shuffle=shuffle, drop_last=drop_last)

    for data_dict in dataloader:
        out_data_dict = {}
        for name, tensor in data_dict.items():
            out_data_dict[name] = data_dict[name].to(device)
        yield out_data_dict

      

A simple perceptron consist of a single linear layer which does the affine transformation for us and than the output of the Linear layer can be provided to a activation function. We are using the sigmoid activation funtion.

In [0]:
# A perceptron classifier
# Lets define a simple perceptron classifier
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm_notebook
class ReviewClassifier(nn.Module):
  "A simple perceptron classifier"
  def __init__(self, num_features):
    super(ReviewClassifier, self).__init__()
    self.fcl = nn.Linear(in_features = num_features, out_features=1)
  
  def forward(self, vectorize_review, apply_sigmoid=False):
    y_out = self.fcl(vectorize_review).squeeze()
    if apply_sigmoid:
      y_out = torch.sigmoid(y_out)
    return y_out

A multilayer perceptron consists of many linear layer and a softmax layer, which is optional. Here we are also using dropout for regularization. dropout generally drops some connections in the neural network so that the network does not overfit.

Neural networks—especially deep networks with a large number of layers—can create interesting coadaptation between the units. “Coadaptation” is a term from neuroscience, but here it simply refers to a situation in which the connection between two units becomes excessively strong at the expense of connections between other units. This usually results in the model overfitting to the data. By probabilistically dropping connections between units, we can ensure no single unit will always depend on another single unit, leading to robust models. Dropout does not add additional parameters to the model, but requires a single hyperparameter—the “drop probability.”18 This, as you might have guessed, is the probability with which the connections between units are dropped. It is typical to set the drop probability to 0.5

Below is the implementation of the MLP with dropout.

In [0]:
class MultiLayerPerceptron(nn.Module):
  "A simple multi Layer perceptron"
  def __init__(self, in_dimension, hidden_dimension, out_dimension):
    super(MultiLayerPerceptron, self).__init__()
    self.fcl1 = nn.Linear(in_dimension, hidden_dimension)
    self.fcl2 = nn.Linear(hidden_dimension, out_dimension)

  def forward(self, x_in, apply_softmax= False):
    "The forward pass for the MLP"
    first_layer_output = self.fcl1(x_in)
    intermediate = F.relu(first_layer_output)
    output = self.fcl2(F.dropout(intermediate, p=0.5))
    if apply_softmax:
      output = F.softmax(output, dim =1)
    return output.squeeze()



## CNN or convolutions neural networks:


In simple terms you convolve a input matrix with a given convolution matrix (also known as kernel matrix). The convolution operation is the element wise multiplication of kernel matrix with input matrix and then taking sum. This gives us an output matrix which contains important information. This important information can be understood in terms of dimensionality of the CNNs. 

**Dimensionality:** The convolutions can be 1D, 2D, and 3d (Conv1d,Conv2d and, Conv3d). The 1D convolution are useful for time serie operations where each timestamp has a input feature vector. In thi way the convolution can learn the important features and sequences in our time series data(can be text or anything). 1D CNNs are most commonly used in NLP operations. 2D are used in computer vision or image processing (as it captures information in both directions= width and heigth of the image). 3D convolutions are used in video processing where the 3rd dimension is for frame in each timestep in the video.

**Channels:** refers to the feature input dimension. For example for 2D convolutions the channels are 3. As pixels correcponds to a feauture in an image, and accross 3 channels these pixels contains important information, similarly "pixels" can be attributed for the words in a text, and channels can be the length of the vocabulary. Pytorch parameters which can be used: in_channel, out_channel

**Kernel Size:** It is the dimension or size of the kernel matrix. In terms of nlp we can say the kernel size refers to n-gram, meaning how much words we wanna keep in context or how many words we are going to look when training the model. larger the kernel , the more words the model will look upon.

**Stride:** controls the step size in the convolution. If the step size is same size of the kernel then the kernel will not overlap with any input data while convolution operation  and will threfor generate a smaller output matrix. Smaller stride will result in larger output.

**Padding:** is done to add extra 0s along the edges of the input. so that it is compatible with the stride while convolving.


**Dilation:** controls how the kernel is applied to the input matrix. A dilation of 2 means that the elements of kernels are 2 position away from each other. It basically introduces hole in the kernel matrix.



The goal of Model here is to determine a configuration of convolution layers that results in the desired feature vector. All CNN applications are like this: there is an initial set of convolutional layers that extract a feature map that becomes input in some upstream processing. In classification, the upstream processing is almost always the application of a Linear (or fc) layer which does the classification for us.

The implementation walk through in this section iterates over the design decisions to construct a feature vector.We begin by constructing an artificial data tensor mirroring the actual data in shape. **The size of the data tensor is going to be three-dimensional**—this is the size of the minibatch of vectorized text data. **If you use a one-hot vector for each character in a sequence of characters, a sequence of one-hot vectors is a matrix, and a minibatch of one-hot matrices is a three-dimensional tensor**. Using the terminology of convolutions, the size of each one-hot vector (usually the size of the vocabulary) is the number of “input channels” and the length of the character sequence is the “width.”


In [0]:
# define a class for the CNNS
class ConvolutionNeuralNetwork(nn.Module):
  def __init__(self, in_channels, num_channels, output_size):
    super(ConvolutionNeuralNetwork, self).__init__()
    self.convet = nn.Sequential(
        nn.Conv1d(in_channels=in_channels, out_channels=num_channels, kernel_size=1),
        nn.ELU(),
        nn.Conv1d(in_channels=num_channels, out_channels=num_channels, kernel_size=1, stride=2),
        nn.ELU(),
        nn.Conv1d(in_channels=num_channels,  out_channels=num_channels, kernel_size=1 , stride=2),
        nn.ELU(),
        nn.Conv1d(in_channels=num_channels, out_channels=num_channels, kernel_size=1),
        nn.ELU()
    )
    self.fcl = nn.Linear(num_channels, output_size)


  def forward(self, x_in, apply_softmax=False):
    features = self.convet(x_in)
    prediction_vector = self.fcl(features)
    if apply_softmax:
      prediction_vector = F.softmax(prediction_vector, dim=2)

    return prediction_vector

        
 

In [0]:
# Lets write  training module to traint the model
# first let define some hyperparameters
from argparse import Namespace

args = Namespace(
    # Data and path information
    frequency_cutoff=25,
    model_state_file='model.pth',
    review_csv='data/yelp/reviews_with_splits_lite.csv',
    save_dir='data/ch3/yelp/',
    vectorizer_file='vectorizer.json',
    # No model hyperparameters
    # Training hyperparameters
    batch_size=256,
    early_stopping_criteria=5,
    learning_rate=0.001,
    num_epochs=5,
    seed=1330,
    cuda = True,
    device = torch.device("cuda")
              )

In [0]:
import torch.optim as optim
def make_train_state(args):
  return {'epoch_index':0,
          'train_loss':[],
          'train_acc':[],
          'val_loss':[],
          'val_acc':[],
          'test_loss':-1,
          'test_acc':1
  }

train_state = make_train_state(args)

In [0]:
if not torch.cuda.is_available():
  args.cuda= False
  args.device = torch.device("cpu")
import json
dataset = ReviewDataset.load_dataset_and_make_vectorizer(args.review_csv)
# dataset.save_vectorizer(args.vectorizer_file)
vectorizer = dataset.get_vectorizer()
# classifier = ReviewClassifier(num_features=len(vectorizer.review_vocab))
classifier = MultiLayerPerceptron(in_dimension=len(vectorizer.review_vocab), hidden_dimension=100, out_dimension=1)
#classifier = ConvolutionNeuralNetwork(in_channels=len(vectorizer.review_vocab),num_channels=256, output_size=1)
classifier = classifier.to(args.device)
loss_func = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(classifier.parameters(), lr = args.learning_rate)

In [61]:
def compute_accuracy(y_pred, y_target):
    y_target = y_target.cpu()
    y_pred_indices = (torch.sigmoid(y_pred)>0.5).cpu().long()#.max(dim=1)[1]
    n_correct = torch.eq(y_pred_indices, y_target).sum().item()
    return n_correct / len(y_pred_indices) * 100


# check if cuda is available or not

np.random.seed(args.seed)
torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed_all(args.seed)

if not torch.cuda.is_available():
    args.cuda = False

print("Using CUDA: {}".format(args.cuda))
args.device = torch.device("cuda" if args.cuda else "cpu")

Using CUDA: True


In [66]:
# method to train and evaluate
for epoch_index in range(args.num_epochs):
  train_state['epoch_index']= epoch_index
  # iterating over the training dataset
  dataset.set_split('train')    
  batch_generator = generate_batches(dataset, batch_size=args.batch_size,
                                     device = args.device)
  running_loss = 0.0
  running_acc = 0.0
  classifier.train()
  for batch_index, batch_dict in enumerate(batch_generator):
    optimizer.zero_grad()
    # for CNN 
    #x_in = batch_dict['x_data'][:,:,None]
    # For other
    x_in = batch_dict['x_data']
    y_pred = classifier(x_in.float())
    loss = loss_func(y_pred, batch_dict['y_target'].float())
    loss_batch = loss.item()
    running_loss += (loss_batch - running_loss) / (batch_index+1)
    loss.backward()
    optimizer.step()
    acc_batch = compute_accuracy(y_pred, batch_dict['y_target'])
    running_acc += (acc_batch- running_acc) / (batch_index+1)
  
  train_state['train_loss'].append(running_loss)
  train_state['train_acc'].append(running_acc)
  average_train_loss = np.mean(train_state['train_loss'])
  average_train_acc = np.mean(train_state['train_acc'])
  print("Current epoch:  {}, Train Loss: {}, and Train Accuracy: {}".format(epoch_index,
                                                              average_train_loss,
                                                              average_train_acc))

  # validation loop for the dataset
  # remember while validation we dont need to calculate gradients
  # and no need to backpropagate
  # so we can directly calculate the accuracy and score using the forward pass
  dataset.set_split('val')
  batch_generator = generate_batches(dataset, batch_size=args.batch_size,
                                    device = args.device)
  running_loss =0.0
  running_acc = 0.0
  classifier.eval()
  for batch_index, batch_dict in enumerate(batch_generator):
    y_pred = classifier(batch_dict['x_data'].float())
    loss = loss_func(y_pred, batch_dict['y_target'].float())
    loss_batch = loss.item()
    running_loss += (loss_batch - running_loss) / (batch_index +1)
    acc_batch = compute_accuracy(y_pred, batch_dict['y_target'])
    running_acc += (acc_batch - running_acc) / (batch_index+1)
    
  train_state['val_loss'].append(running_loss)
  train_state['val_acc'].append(running_acc)
  average_val_loss= np.mean(train_state['val_loss'])
  average_val_acc = np.mean(train_state['val_acc'])

  print("Current epoch: {}, Val Loss: {} and, Val Accuracy: {}".format(epoch_index,
                                                               average_val_loss,
                                                              average_val_acc))

Current epoch:  0, Train Loss: 0.3343543533990584, and Train Accuracy: 87.21660539215691
Current epoch: 0, Val Loss: 0.23227112879976627 and, Val Accuracy: 91.24755859375
Current epoch:  1, Train Loss: 0.26143680106288464, and Train Accuracy: 90.06459354575168
Current epoch: 1, Val Loss: 0.21898904116824267 and, Val Accuracy: 91.69921875
Current epoch:  2, Train Loss: 0.22520814279260717, and Train Accuracy: 91.42667483660132
Current epoch: 2, Val Loss: 0.2157269430657228 and, Val Accuracy: 91.77652994791667
Current epoch:  3, Train Loss: 0.2012813241385362, and Train Accuracy: 92.33940972222223
Current epoch: 3, Val Loss: 0.21489432116504759 and, Val Accuracy: 91.7144775390625
Current epoch:  4, Train Loss: 0.18316233113310695, and Train Accuracy: 93.04636437908498
Current epoch: 4, Val Loss: 0.21683721505105497 and, Val Accuracy: 91.728515625


In [67]:
# Evaluation on the held out dataset
dataset.set_split('test')
batch_generator = generate_batches(dataset, batch_size=args.batch_size, 
                                    device= args.device)
runnning_loss = 0.0
running_acc = 0.0
classifier.eval()
print("testing")
for batch_index, batch_dict in enumerate(batch_generator):
  # print(batch_dict)
  y_pred = classifier(batch_dict['x_data'].float())
  loss = loss_func(y_pred, batch_dict['y_target'].float())
  loss_batch = loss.item()
  running_loss += (loss_batch - running_loss) / (batch_index+1)
  acc_batch = compute_accuracy(y_pred, batch_dict['y_target'])
  running_acc += (acc_batch- running_acc) / (batch_index+1)

train_state['test_loss'] = running_loss
train_state['test_acc'] = running_acc
average_test_loss = train_state['test_loss']
average_test_acc = train_state['test_acc']
print("Test Loss: {}, and Test Accuracy: {}".format(average_test_loss,
                                                              average_test_acc)) 

testing
Test Loss: 0.22730396315455437, and Test Accuracy: 91.61376953125


In [0]:
# inference and classifying new data points
def predict_rating(review, classifier, vectorizer, decision_threshold=0.5):
  "Predict the rating of the review"
  "decision boundary is the threshold which separates the 2 ratings"
  review = preprocess_data(review)
  vectorized_review = torch.tensor(vectorizer.vectorize(review=review)).to(args.device)

  result = classifier(vectorized_review.view(1,-1))
  probability = torch.sigmoid(result).item()
  print(probability)
  index=1
  if probability < decision_threshold:
    index=0
  return vectorizer.rating_vocab.lookup_index(index)

In [69]:
new_review = "That dinner was not aweful"
prediction = predict_rating(new_review,classifier,vectorizer)
print("{} --> {}".format(new_review,prediction))

new_review = "That burger was very bad"
prediction = predict_rating(new_review,classifier,vectorizer)
print("{} --> {}".format(new_review,prediction))

new_review = "That burger was very good"
prediction = predict_rating(new_review,classifier,vectorizer)
print("{} --> {}".format(new_review,prediction))

0.08916141092777252
That dinner was not aweful --> negative
0.35558733344078064
That burger was very bad --> negative
0.7931451797485352
That burger was very good --> positive


In [70]:
# weight inspection to see most common positive and negative words
fcl_weights = classifier.fcl1.weight.detach()[0]
_, indices = torch.sort(fcl_weights, dim=0, descending=True)
indices = indices.cpu()
indices = indices.numpy().tolist()
print("Influential words in Positive Reviews")
print("--------------------------------------")
for i in range(20):
  print(vectorizer.review_vocab.lookup_index(indices[i]))

Influential words in Positive Reviews
--------------------------------------
horrible
worst
bland
mediocre
awful
dried
unfriendly
slowest
nmaybe
disappointing
meh
disgusting
poorly
unacceptable
underwhelmed
terrible
tasteless
eh
mills
rude


In [71]:
# Top 20 negative words
print("Influential words in Negative Reviews:")
print("--------------------------------------")
indices.reverse()
for i in range(20):
    print(vectorizer.review_vocab.lookup_index(indices[i]))

Influential words in Negative Reviews:
--------------------------------------
fantastic
delicious
pho
amazing
vegas
perfection
pleasantly
nyes
penny
yummy
ngreat
magic
moist
divine
variety
chill
melt
perfect
heaven
chinatown
